# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [86]:
!pip install langchain_community langchain_experimental langchain_openai

In [87]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [88]:
 %pip install faiss-cpu > /dev/null
 %pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

In [90]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [ ]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [ ]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [ ]:
OBJECTIVE = "Write a weather report for SF today"

In [ ]:
verbose = False
max_iterations: Optional[int] = 3

# Instantiate the BabyAGI with your custom objective and tools
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,  # This should contain the embedded sonnet documents
    task_execution_chain=agent_executor,  # AgentExecutor using your updated tools (e.g., sonnets_of_shakespeare)
    verbose=verbose,
    max_iterations=max_iterations,
)


In [ ]:
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I should use the TODO action to create a todo list
Action: TODO
Action Input: "Create a todo list for writing a weather report for SF today"
Observation: 

1. Gather all necessary information: Start by collecting all the necessary information for writing a weather report for San Francisco today. This includes current temperature, humidity, wind speed, and any other relevant data.

2. Check weather forecast: Before writing the report, check the weather forecast for San Francisco today to get an idea of what to expect. This will help you accurately report on the current weather conditions.

3. Determine target audience: Consider who will be reading the weather report and tailor your writing style and language accordingly. A report for a local news station may be different from one for a scientific journal.

4. Create an outline: Organize your thoughts and i

{'objective': 'Write a weather report for SF today'}

In [91]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

# Wellness planner prompt
wellness_prompt = PromptTemplate.from_template(
    "You are a certified wellness coach who creates personalized 5-day fitness and wellness plans based on a person's goal. Create a 5-day plan for this objective: {objective}. Respond in the following format:\nAction: <action to take>\nAction Input: <input to the action>"
)

wellness_chain = LLMChain(llm=OpenAI(temperature=0), prompt=wellness_prompt)

# Search tool for real-time info (e.g., local gyms, meal plans, current trends)
search = SerpAPIWrapper()

# Tools setup
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for looking up healthy meal options, fitness trends, local gyms or classes, or any wellness-related current info",
    ),
    Tool(
        name="WellnessPlanner",
        func=wellness_chain.run,
        description="useful for generating 5-day fitness and wellness routines based on goals like weight loss, stress reduction, or strength building",
    ),
]

# Prompt setup
prefix = """You are an AI wellness advisor helping someone achieve their fitness or lifestyle goals. Your job is to build a plan or assist based on the objective and what has already been done: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)


# Instantiate the BabyAGI with your custom objective and tools, and handle parsing errors
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [92]:
OBJECTIVE = "Create a 5-day fitness and wellness plan for stress reduction and improved sleep"


In [93]:
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: To create a todo list, I need to understand the person's goals and priorities.
Action: WellnessPlanner
Action Input: Goals and priorities
Observation: 

Day 1:
Action: Set Goals and Priorities
Action Input: Meet with client to discuss their current goals and priorities and help them identify any areas they would like to focus on.

Day 2:
Action: Create a Vision Board
Action Input: Have client create a vision board with images and words that represent their goals and priorities. This will serve as a visual reminder and motivation for them throughout the week.

Day 3:
Action: Schedule Time for Goal Work
Action Input: Help client schedule specific times throughout the week to work on their goals and priorities. This will ensure they make time for these important tasks and stay on track.

Day 4:
Action: Prioritize Tasks
Action Input: Have client make a list o

{'objective': 'Create a 5-day fitness and wellness plan for stress reduction and improved sleep'}